In [7]:
import pandas as pd
import re

In [8]:
# Functions

def parse_bible_text(pageContent):
    """
    Parse bible pageContent and split it into separate entries based on verse numbers.
    
    This function looks for patterns like:
    - 4:1 (chapter:verse at beginning)
    - 2( or 3h or 4A etc. (verse number followed by letter)
    
    Args:
        pageContent (str): The raw bible pageContent to parse
        
    Returns:
        list: List of dictionaries containing parsed verses
    """
    
    # Remove extra whitespace and normalize pageContent
    pageContent = re.sub(r'\s+', ' ', pageContent.strip())
    
    # Find all verse markers - looking for number followed by letter or punctuation
    # Pattern explanation:
    # \b(\d+) - word boundary followed by one or more digits (verse number)
    # (?=[a-zA-Z(]) - positive lookahead for letter or opening parenthesis
    verse_pattern = r'\b(\d+)(?=[a-zA-Z(])'
    
    # Also look for chapter:verse pattern at the beginning
    chapter_verse_pattern = r'\b(\d+:\d+)\s'
    
    # Find all matches
    verse_matches = list(re.finditer(verse_pattern, pageContent))
    chapter_matches = list(re.finditer(chapter_verse_pattern, pageContent))
    
    # Combine and sort all matches by position
    all_matches = []
    
    # Add chapter:verse matches
    for match in chapter_matches:
        all_matches.append({
            'start': match.start(),
            'end': match.end(),
            'verse_ref': match.group(1),
            'type': 'chapter_verse'
        })
    
    # Add verse-only matches
    for match in verse_matches:
        all_matches.append({
            'start': match.start(),
            'end': match.start() + len(match.group(1)),  # Only include the number, preserve letter
            'verse_ref': match.group(1),
            'type': 'verse'
        })
    
    # Sort by position
    all_matches.sort(key=lambda x: x['start'])
    
    # Split text based on matches
    parsed_verses = []
    
    if not all_matches:
        # No verses found, return the whole pageContent
        return [{'verse_ref': 'unknown', 'pageContent': pageContent.strip(), 'type': 'unknown'}]
    
    # Handle pageContent before first verse (usually heading)
    if all_matches[0]['start'] > 0:
        heading_pageContent = pageContent[:all_matches[0]['start']].strip()
        if heading_pageContent:
            parsed_verses.append({
                'verse_ref': 'heading',
                'pageContent': heading_pageContent,
                'type': 'heading'
            })
    
    # Process each verse
    for i, match in enumerate(all_matches):
        # Determine where this verse ends
        if i + 1 < len(all_matches):
            verse_end = all_matches[i + 1]['start']
        else:
            verse_end = len(pageContent)
        
        # Extract verse pageContent
        verse_text = pageContent[match['end']:verse_end].strip()
        
        if verse_text:  # Only add if there's actual content
            parsed_verses.append({
                'verse_ref': match['verse_ref'],
                'pageContent': verse_text,
                'type': match['type']
            })
    
    return parsed_verses


def clean_csv_data(df, text_column='pageContent'):
    """
    Apply the parsing function to a pandas DataFrame.
    
    Args:
        df (DataFrame): The input DataFrame
        text_column (str): Name of the column containing the text to parse
        
    Returns:
        DataFrame: New DataFrame with parsed verses
    """
    
    all_parsed = []
    
    for index, row in df.iterrows():
        text = str(row[text_column])
        parsed_verses = parse_bible_text(text)
        
        for verse in parsed_verses:
            new_row = row.copy()
            new_row['original_index'] = index
            new_row['verse_ref'] = verse['verse_ref']
            new_row['parsed_text'] = verse['pageContent']
            new_row['verse_type'] = verse['type']
            all_parsed.append(new_row)
    
    return pd.DataFrame(all_parsed)


# # Replace special characters like â€œ with proper quotes
# def clean_special_characters(text):
#     """
#     Enhanced function to clean common encoding issues in text.
#     This version handles more Unicode variations and uses regex for better matching.
#     """
#     if pd.isna(text) or text == '':
#         return text
    
#     text = str(text)
    
#     # Define replacements with more comprehensive patterns
#     replacements = [
#         # Various quote patterns
#         (r'â€œ', '"'),  # Left double quote
#         (r'â€', '"'),   # Right double quote  
#         (r'â€™', "'"),  # Right single quote
#         (r'â€˜', "'"),  # Left single quote
#         (r'â€"', '—'),  # Em dash
#         (r'â€"', '–'),  # En dash
#         (r'â€¦', '...'), # Ellipsis
        
#         # Additional common encoding issues
#         (r'Â ', ' '),    # Non-breaking space issues
#         (r'Â', ''),      # Standalone Â characters
        
#         # Handle any remaining â followed by euro and other chars (fallback)
#         (r'â€[œ"]', '"'),  # Various quote combinations
#         (r'â€[™˜]', "'"),  # Various apostrophe combinations
#         (r'â€[—–]', '—'),  # Various dash combinations
#     ]
    
#     # Apply all replacements
#     for pattern, replacement in replacements:
#         text = re.sub(pattern, replacement, text)
    
#     return text


def clean_special_characters(text):
    """
    Cleans special characters from the text.
    """
    if pd.isna(text) or text == '':
        return text
    
    # Define a regex pattern to match common special characters
    pattern = r'[^\w\s,.!?;:\'\"-]'
    
    # Replace special characters with an empty string
    cleaned_text = re.sub(pattern, '', text)
    
    return cleaned_text


def has_verse_numbers(text):
    """Check if text contains verse number patterns"""
    if pd.isna(text) or text.strip() == '':
        return False
        
    text = str(text)
    
    # Check for chapter:verse pattern (e.g., "4:1")
    chapter_verse_pattern = r'\b\d+:\d+\b'
    if re.search(chapter_verse_pattern, text):
        return True
        
    # Check for verse number followed by letter or punctuation (e.g., "2But", "3(")
    verse_pattern = r'\b\d+(?=[a-zA-Z(])'
    if re.search(verse_pattern, text):
        return True
        
    return False


def consolidate_rows_without_verses(df, text_column='pageContent'):
    """
    Consolidate rows by appending text from rows without verse numbers 
    to the previous row.
    
    Args:
        df (DataFrame): The input DataFrame
        text_column (str): Name of the column containing the text to check
        verse_column (str): Name of the column containing verse references
        
    Returns:
        DataFrame: New DataFrame with consolidated rows
    """
    
    def has_verse_numbers(text):
        """Check if text contains verse number patterns"""
        if pd.isna(text) or text.strip() == '':
            return False
            
        text = str(text)
        
        # Check for chapter:verse pattern (e.g., "4:1")
        chapter_verse_pattern = r'\b\d+:\d+\b'
        if re.search(chapter_verse_pattern, text):
            return True
            
        # Check for verse number followed by letter or punctuation (e.g., "2But", "3(")
        verse_pattern = r'\b\d+(?=[a-zA-Z(])'
        if re.search(verse_pattern, text):
            return True
            
        return False
    
    # Create a copy of the dataframe to work with
    consolidated_df = df.copy()
    consolidated_rows = []
    
    i = 0
    while i < len(consolidated_df):
        current_row = consolidated_df.iloc[i].copy()
        current_text = str(current_row[text_column])
        
        # Check if current row has verse numbers
        if has_verse_numbers(current_text):
            # This row has verse numbers, keep it as is
            consolidated_rows.append(current_row)
        else:
            # This row doesn't have verse numbers
            if len(consolidated_rows) > 0:
                # Append to the previous row
                previous_row = consolidated_rows[-1]
                previous_text = str(previous_row[text_column])
                
                # Combine the text with a space
                combined_text = previous_text + " " + current_text
                previous_row[text_column] = combined_text
                
                print(f"Consolidated row {i}: '{current_text[:50]}...' -> appended to previous row")
            else:
                # No previous row to append to, keep this row
                consolidated_rows.append(current_row)
                print(f"Row {i}: No previous row to append to, keeping as standalone")
        
        i += 1
    
    # Convert back to DataFrame
    result_df = pd.DataFrame(consolidated_rows)
    result_df.reset_index(drop=True, inplace=True)
    
    return result_df


# Append pageContent values to the previous row when they have a verse_ref that is 'heading' or 'unknown'
def consolidate_rows_mid_section(df, text_column='pageContent'):
    """
    Consolidate rows by appending text from rows without verse numbers 
    to the previous row.
    
    Args:
        df (DataFrame): The input DataFrame
        text_column (str): Name of the column containing the text to check
        
    Returns:
        DataFrame: New DataFrame with consolidated rows
    """
    
    consolidated_df = df.copy()
    consolidated_rows = []
    
    i = 0
    while i < len(consolidated_df):
        current_row = consolidated_df.iloc[i].copy()
        current_text = str(current_row[text_column])
        
        # Check if current row has a verse_ref that is 'heading' or 'unknown'
        if current_row.get('verse_ref') in ['heading', 'unknown']:
            # This row is a heading or unknown, append to previous row
            if len(consolidated_rows) > 0:
                previous_row = consolidated_rows[-1]
                previous_text = str(previous_row[text_column])
                
                # Combine the text with a space
                combined_text = previous_text + " " + current_text
                previous_row[text_column] = combined_text
                
                print(f"Consolidated row {i}: '{current_text[:50]}...' -> appended to previous row")
            else:
                # No previous row to append to, keep this row
                consolidated_rows.append(current_row)
                print(f"Row {i}: No previous row to append to, keeping as standalone")
        else:
            # This row has a valid verse_ref, keep it as is
            consolidated_rows.append(current_row)
        
        i += 1
    
    # Convert back to DataFrame
    result_df = pd.DataFrame(consolidated_rows)
    result_df.reset_index(drop=True, inplace=True)
    
    return result_df


def combine_rows_by_condition(data):
    """
    Combines rows in a table when the first word starts with a lowercase letter or number.
    
    Parameters:
    data: Can be a list of strings, pandas DataFrame, or list of lists
    
    Returns:
    Combined data in the same format as input
    """
    
    # Convert input to list of strings for processing
    if isinstance(data, pd.DataFrame):
        # If DataFrame, convert to list of strings (assuming single column or join columns)
        if data.shape[1] == 1:
            string_list = data.iloc[:, 0].tolist()
        else:
            # Join multiple columns with space
            string_list = data.apply(lambda row: ' '.join(row.astype(str)), axis=1).tolist()
        is_dataframe = True
    elif isinstance(data, list) and len(data) > 0 and isinstance(data[0], list):
        # List of lists - join each sublist
        string_list = [' '.join(map(str, row)) for row in data]
        is_dataframe = False
        is_list_of_lists = True
    else:
        # Assume list of strings
        string_list = data.copy()
        is_dataframe = False
        is_list_of_lists = False
    
    if not string_list:
        return data
    
    combined_list = []
    
    for i, current_row in enumerate(string_list):
        current_row = str(current_row).strip()
        
        if not current_row:
            combined_list.append(current_row)
            continue
            
        # Get the first word
        words = current_row.split()
        if not words:
            combined_list.append(current_row)
            continue
            
        first_word = words[0]
        
        # Check if first character is lowercase or digit
        first_char = first_word[0]
        should_combine = first_char.islower() or first_char.isdigit()
        
        if should_combine and combined_list:
            # Append to previous row with a space
            combined_list[-1] = combined_list[-1] + " " + current_row
        else:
            # Add as new row
            combined_list.append(current_row)
    
    # Convert back to original format
    if isinstance(data, pd.DataFrame):
        if data.shape[1] == 1:
            return pd.DataFrame(combined_list, columns=data.columns)
        else:
            # For multi-column DataFrames, return as single column
            return pd.DataFrame(combined_list, columns=['combined_text'])
    elif is_list_of_lists:
        # Convert back to list of lists (single item lists)
        return [[item] for item in combined_list]
    else:
        return combined_list
    

# Remove all instances of '[number]' in a text string.

def remove_square_brackets(text):
    """
    Remove all instances of '[number]' in a text string.
    
    Args:
        text (str): The input text
        
    Returns:
        str: Text with all '[number]' patterns removed
    """
    if pd.isna(text):
        return text
    
    # Use regex to remove patterns like [1], [2], etc.
    return re.sub(r'\[\d+\]', '', str(text)).strip()



In [9]:
# Variables

# Import data
import_filename = 'ESV_extracted_text.csv'
import_filepath = "C:\\Users\\hlmq\\code\\bible-app\\Process ESV\\Out\\"
# Export data
export_filename = 'ESV_Bible_Extracted_and_Parsed.csv'
export_filepath = "C:\\Users\\hlmq\\code\\bible-app\\Process ESV\\Out\\"

In [10]:
# Import Data
df = pd.read_csv(str(import_filepath)+str(import_filename))

# Delete rows with null values in 'pageContent' column
mask = df['pageContent'].isnull()
df = df[~mask]

df.reset_index(drop=True, inplace=True)

df.head()

,docName,pageContent,pageNumber
0,ESV Bible.pdf,JOHN\nChapter 1\nChapter 2\nChapter 3\nChapter...,4996
1,ESV Bible.pdf,The Word Became Flesh\n1:1 In the beginning wa...,4998
2,ESV Bible.pdf,"right to become children of God, 13who were bo...",4999
3,ESV Bible.pdf,The Testimony of John the Baptist\n19And this ...,5000
4,ESV Bible.pdf,baptizing.,5001


In [11]:
# Apply character cleaning first
if 'pageContent' in df.columns:
    df['pageContent'] = df['pageContent'].apply(clean_special_characters)
    df['pageContent'] = df['pageContent'].apply(remove_square_brackets)
    content = combine_rows_by_condition(df['pageContent'].tolist())
    df = pd.DataFrame(content, columns=['pageContent'])
    df = clean_csv_data(df, text_column='pageContent')

print(f"\nParsing complete. Expanded from {len(df)} to {len(df)} rows")


Parsing complete. Expanded from 1154 to 1154 rows


In [12]:
output_file = str(export_filepath) + str(export_filename)
df.to_csv(output_file, index=False)
print(f"\n=== EXPORT ===")
print(f"Cleaned data saved to: {output_file}")


=== EXPORT ===
Cleaned data saved to: C:\Users\hlmq\code\bible-app\Process ESV\Out\ESV_Bible_Extracted_and_Parsed.csv
